<a href="https://colab.research.google.com/github/amankumar-ds/COVID19_pattern_analysis/blob/main/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [4]:
from google.colab import files 
uploaded = files.upload()

Saving Bengaluru_House_Data.csv to Bengaluru_House_Data.csv


In [12]:
import io 
  
df = pd.read_csv(io.BytesIO(uploaded['Bengaluru_House_Data.csv'])) 
print(df.head())

              area_type   availability  ... balcony   price
0  Super built-up  Area         19-Dec  ...     1.0   39.07
1            Plot  Area  Ready To Move  ...     3.0  120.00
2        Built-up  Area  Ready To Move  ...     3.0   62.00
3  Super built-up  Area  Ready To Move  ...     1.0   95.00
4  Super built-up  Area  Ready To Move  ...     1.0   51.00

[5 rows x 9 columns]


In [21]:
df.columns

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')

In [34]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) ==2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [35]:
df2 = df.copy()
df2['total_sqft'] = df2['total_sqft'].apply(convert_sqft_to_num)

In [36]:
X = df2.iloc[:,[0,1,2,3,4,5,6,7]].values
y = df2.iloc[:, 8].values

In [37]:
from sklearn.preprocessing import LabelEncoder

le_X_0= LabelEncoder()
le_X_1= LabelEncoder()
le_X_2= LabelEncoder()
le_X_3= LabelEncoder()
le_X_4= LabelEncoder()



X[:, 0] = le_X_0.fit_transform(X[:, 0])
X[:, 1] = le_X_1.fit_transform(X[:, 1])
X[:,2] =  le_X_2.fit_transform(X[:,2].astype(str))
X[:, 3] = le_X_3.fit_transform(X[:, 3].astype(str))
X[:, 4] = le_X_4.fit_transform(X[:, 4].astype(str))

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [39]:
from xgboost import XGBRegressor
regressor  = XGBRegressor()

In [40]:
regressor.fit(X_train, y_train)

[09:19:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [41]:
Y_pred_train = regressor.predict(X_train)

In [42]:
y_pred = regressor.predict(X_test)

In [45]:
def rmsle(y_pred,y_test) :
 error = np.square(np.log10(y_pred +1) - np.log10(y_test +1)).mean() ** 0.5
 Acc = 1 - error
 return Acc

print("Accuracy attained on Training Set = ",rmsle(Y_pred_train, y_train))
print("Accuracy attained on Test Set = ",rmsle(y_pred,y_test))

Accuracy attained on Training Set =  0.8437178847274824
Accuracy attained on Test Set =  0.8325375220827818
